# Лабораторная работа №3
## Выполнил студент группы БВТ2001 Ильинская Вера

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [ ]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [ ]:
#смещение символов по префиксам через массив Пи

stroka = str (input("Введите строку: "))
pod_str = str (input("Введите подстроку: "))

#формирование массива Пи, хранящего макс.длины суфиксов, совпадающих с префи-ми
#суфикс - все возможные варианты после префикса 
def createMasPi(pod_str): 
  dictPi = [0] * len(pod_str) #массив Пи длинной, как образ (подстрока)
  j = 0
  i = 1
  while (i<len(pod_str)):
    if (pod_str[j] == pod_str[i]): #если символы образа совпали
      dictPi[i] = j + 1
      #сдвигаем счётчики j и i на 1
      j += 1
      i += 1
    else: #если символы образа не совпали
      if (j == 0):
        dictPi[i] = 0
        i += 1 #смещаём i к следующему элементу
      else:
        j = dictPi[j-1] #сдвиг j назад для нового сравнения
  return(dictPi)


def findPodstrKMP(stroka, pod_str, ignoreCase):
  #в том случае, если нам нужно игнорировать регистр
  if (ignoreCase):
    stroka = stroka.lower()
    pod_str = pod_str.lower()

  dictPi = createMasPi(pod_str)
  stro_len = len(stroka)
  pod_len = len(pod_str)

  i = 0
  j = 0
  #пока не пройдёмся по всей строке
  while (i<stro_len):
    flag = False
    if (stroka[i] == pod_str[j]):
      i += 1
      j += 1
      if (j == pod_len): #если полное совпадение образа и подстроки строки
        print(f"Индекс образа: {i-pod_len}")
        flag = True #для регистров и отслеживания нахождения
        break
    else:
      if (j > 0):
        j = dictPi[j-1] #присваиваем j смещение из массива макс.длинн суфиксов
      else:
        i += 1 #если j равен 0, то сдвигаемся по строке на 1
  if (i == stro_len and j != pod_len):
    print("Образ не был найден")


findPodstrKMP(stroka, pod_str, True)

Введите строку: Шла Саша по шоссе
Введите подстроку: саша
Индекс образа: 4


#### Упрощенный алгоритм Бойера-Мура

In [ ]:
#поиск с помощью таблицы смещений символов подстроки

stroka = str (input("Введите строку: "))
pod_str = str (input("Введите подстроку: "))

#метод формирования таблицы смещения для подстроки (образа)
def createBiasTabl(pod_str):
  setList = set() #создаём список для уникальных значений (которые раньше не встречались)
  N = len(pod_str) #число символов в подстроке (образе)
  dictSym = defaultdict() #"таблица" смещений

  #определяем смещение от предпоследнего символа до первого
  for i in range(N-2, -1, -1):
    #если такой символ уже был рассмотрен, то в 
    #в dict не добавляем
    if (pod_str[i] not in setList):
      dictSym[pod_str[i]] = N-i-1 #добавляем в словарь символ как ключ = смещение
      setList.add(pod_str[i]) #добавляем символ для будущей проверки повторяющихся сим-в

  #если последнего символа не встречался до этого в образе, то его смещение = длине образа
  #если последний символ уже встречался, то его смещение итак = смещению его "сородича"
  if (pod_str[N-1] not in setList):
    dictSym[pod_str[N-1]] = N

  dictSym['*'] = N #если в будущем проверяемый символ не входит в таблицу смещения, то
                   #с помощью "*" смещаем на длину подслова
  return (dictSym)


def findPodstrBM(stroka, pod_str, ignoreCase):
  #в том случае, если нам нужно игнорировать регистр
  if (ignoreCase):
    stroka = stroka.lower()
    pod_str = pod_str.lower()

  dictSym = createBiasTabl(pod_str)
  stro_len = len(stroka)
  pod_len = len(pod_str)
  
  if (stro_len >= pod_len):
    i = pod_len - 1 #определяем индекс строки для начала проверки
                    #в начале индекс строки = индексу последнего символа подстроки
    while (i < stro_len):
       k = 0 #с помощью k идём от индекса i по выбранному куску 
             #строки для проверки совпадения с подстрокой
       flag = False
       #пробегаем в подстроке от последнего символа до первого
       for j in range(pod_len-1, -1, -1):
         #попарно сравниваем символы образа и проверяемого куска строки
         #если символ не совпал с символом образа
         if (stroka[i-k] != pod_str[j]):
            if (j == (pod_len-1)): #если не совпал последний символ образа
                #если в dictSym есть такой символ, то bias = смещению символа, иначе равен длине образа
                bias = dictSym[stroka[i]] if dictSym.get(stroka[i], False) else dictSym['*']
            else: #если не совпал не последний символ образа
                bias = dictSym[pod_str[j]] #определяем смещение по символу образа

            i += bias
            flag = True #для регистров и отслеживания нахождения
            break
         k += 1

       if (not flag):
         print(f"Индекс образа: {i-k+1}")
         break
    else:
      print("Образ не был найден") 
  #если подстрока больше, чем сама строка
  else:
    print("Образ невозможно найти")

findPodstrBM(stroka, pod_str, True)

Введите строку: Корабли всё лавировали
Введите подстроку: ЛАВИРОВАЛИ
Индекс образа: 12


####Использование всех методов поиска подстроки, расчёт времени работы их работы и сравнение со временем стандартной функции поиска



In [ ]:
from datetime import datetime
import time

stroka = str (input("Введите строку: "))
pod_str = str (input("Введите подстроку: "))
print("\n")

#расчёт времени работы стандарной функции поиска
start_time = time.perf_counter() 

stroka.find(pod_str)

final_time = time.perf_counter()
work_time_standart = (final_time-start_time)/0.10
print(f"Время работы стандарной функции поиска - {work_time_standart} секунд \n")

#расчёт времени работы поиска методом Кнута-Морриса-Пратта
start_time = time.perf_counter() 

findPodstrKMP(stroka, pod_str, False)

final_time = time.perf_counter()
work_time_sel = (final_time-start_time)/0.10
print(f"Время работы поиска методом Кнута-Морриса-Пратта  - {work_time_sel} секунд \n")

#расчёт времени работы поиска методом Бойера-Мура
start_time = time.perf_counter() 

findPodstrBM(stroka, pod_str, False)

final_time = time.perf_counter()
work_time_sel = (final_time-start_time)/0.10
print(f"Время работы поиска методом Бойера-Мура  - {work_time_sel} секунд\n")

print("Можем сделать вывод, что все методы поиска по времени проигрывают стандартной функции поиска")

Введите строку: Ехал Грека через реку
Введите подстроку: Грека


Время работы стандарной функции поиска - 0.000385799999946812 секунд 

Индекс образа: 5
Время работы поиска методом Кнута-Морриса-Пратта  - 0.0009429000002114662 секунд 

Индекс образа: 5
Время работы поиска методом Бойера-Мура  - 0.0010403999999653024 секунд

Можем сделать вывод, что все методы поиска по времени проигрывают стандартной функции поиска


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [ ]:
from queue import PriorityQueue #для инициализации объекта очереди для нахождения мин.значений ячеек

#класс, определяющий параметры заданного набора
class Pos:
    #принимаем позицию и дистанцию, от которой элемент начал (дабы понять, по чему он прошёл)
    def __init__(self, position, startDist):
        self.position = position
        self.startDist = startDist

    #метод сравнения ячеек (меньше ли)
    def __lt__(self, other):
        return self.startDist < other.startDist

    #метод для вывода массива start в виде поля 4 на 4
    def __str__(self):
        return '\n'.join((4*'{:3}').format(*[i%(4*4) for i in self.position[i:]]) 

#выводим цифры в виде поля 4 на 4
for i in range(0, 16, 4))
solved = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 5, 4, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]


#метод определения возможности нахождения конечной последовательности
def finOrder(switch):
    #seenEle - массив пройденных ячеек
    #cycles -  кол-во циклов прохождения по массиву ячеек
    seenEle = set()
    cycles = 0
    for i in switch:
        #добавление в массив пройденных ячеек полей, которых нет 
        if (i not in seenEle):
            cycles += 1
            while (i not in seenEle):
                seenEle.add(i) #добавляем значение в массив 
                i = switch[i] #переход на это значение
    #определяем, можно ли из данной последовательности составить правильную финальную последовательность
    return (cycles + len(switch)) % 2 


#проверка возможности нахождения решения
if (finOrder(start) == 0):
    print('Невозможно составить последовательность')
else:
    #заполняем кортеж значениями поля
    start = tuple(start)
    start_pos = Pos(start, 0)

    #выводим поле 4 на 4
    print("Начальная матрица пятнашек: ")
    print(start_pos)
    print()

    queue = PriorityQueue() #объект очереди, с помощью которого будем доставать минимальные значения ячеек
    queue.put(start_pos) #кладем в очередь с приоритетом первоначальную позицию (по возрастанию)

    #массив пройденных ячеек
    seenEle  = set([start_pos])
    
    #словарь ячеек для дальнейшего перемещения
    came_from = {start_pos.position: None}
    
    while (start_pos.position != solved):
        #извлекаем из очереди позицию с наименьшим приоритетом
        start_pos = queue.get()
        #кладем в очередь все соседние позиции и повторяем, пока не вытащим конечную позицию из очереди
        for i in moves(start_pos.position):
            if i not in seenEle :
                queue.put(Pos(i, start_pos.startDist + 1))
                came_from[i] = start_pos
                seenEle .add(i)

    #решение (путь)
    solution = []
    #сохраняем конечную позицию
    final_pos = start_pos
    #идем в обратном порядке и запоминаем очередность хода
    while (start_pos.position != start):
        #запоминаем ход
        start_pos = came_from[start_pos.position]
        number = start_pos.position[final_pos.position.index(0)]
        solution.append(number)
        final_pos = start_pos
    solution.reverse()

    print("Последовательность передвигаемых номеров: ")
    print(solution)


#метод определения движения ячейки
def moves(position):
    first_pos = position.index(0)
    #возвращаем значения из списка = целое значение и остаток
    i, j = divmod(first_pos, 4)
    offsets = [] #массив смещений
    #для движения вниз
    if (i > 0): 
      offsets.append(-4)     
    #для движения вверх
    if (i < 3): offsets.append(4)  
    #для движения вправо
    if (j > 0): offsets.append(-1)
    #для движения влево
    if (j < 3): offsets.append(1)  
    #идём по всем смещениям и определяем, как должны двигаться
    for offset in offsets:
        #заполянем пустую ячейку элементом, который мы двигаем
        emp_cell = first_pos + offset
        #если ячейка не последняя, то нужно вывести её индекс
        #если последняя, то выводим индекс элемента перед ней
        #создаём кортеж
        yield tuple(position[emp_cell] if x == first_pos else position[first_pos] if x == emp_cell else position[x] for x in range(16))

Невозможно составить последовательность
